In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import warnings
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import platform

warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [4]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE': 1024,
    'SEED':42
}

PATH = os.getcwd() + '/data/'
if platform.system() == 'Darwin':
    LOADPATH = '/Users/a1r/Desktop/DL/timeseries_new_data/'
else:
    LOADPATH = '/home/a1r/바탕화면/DL/timeseries_new_data/'

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [6]:
train_input = np.load(LOADPATH + 'train_input_minmax.npy')
train_target = np.load(LOADPATH + 'train_target_minmax.npy')

In [7]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [8]:
data_len = len(train_input)
train_dataset = CustomDataset(train_input[:-int(data_len*0.2)], train_target[:-int(data_len*0.2)])
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(train_input[-int(data_len*0.2):], train_target[-int(data_len*0.2):])
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### LSTM

In [9]:
class LSTM(nn.Module):
    def __init__(self, input_size=9, hidden_size=512, output_size=CFG['PREDICT_SIZE']):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers = 2, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(hidden_size//2, output_size)
        )
            
        self.actv = nn.ReLU()
    
    def forward(self, x):
        # x shape: (B, TRAIN_WINDOW_SIZE, 9)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)
        
        # LSTM layer
        lstm_out, hidden = self.lstm(x, hidden)
        
        # Only use the last output sequence
        last_output = lstm_out[:, -1, :]
        
        # Fully connected layer
        output = self.actv(self.fc(last_output))
        
        return output.squeeze(1)
    
    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(2, batch_size, self.hidden_size, device=device),
                torch.zeros(2, batch_size, self.hidden_size, device=device))

In [10]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None
    train_loss = {}
    val_loss = {}
    
    # Epoch
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss_li = []
        train_mae = []
    
    # Iteration
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss_li.append(loss.item())
        
        val_loss_li = validation(model, val_loader, criterion, device)
        val_loss_mean = np.mean(val_loss_li)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss_li):.5f}] Val Loss : [{val_loss_mean:.5f}]')
        
        if best_loss > val_loss_mean:
            best_loss = val_loss_mean
            best_model = model
            print('Model Saved')

        train_loss[epoch] = train_loss_li
        val_loss[epoch] = val_loss_li

    return best_model, train_loss, val_loss

In [11]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return val_loss

In [12]:
model = LSTM()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model, train_loss, val_loss = train(model, optimizer, train_loader, val_loader, device)

  0%|          | 0/4010 [00:00<?, ?it/s]

  0%|          | 0/1003 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.02996] Val Loss : [0.03218]
Model Saved


  0%|          | 0/4010 [00:00<?, ?it/s]

  0%|          | 0/1003 [00:00<?, ?it/s]

...

  0%|          | 0/1003 [00:00<?, ?it/s]

Epoch : [9] Train Loss : [0.01691] Val Loss : [0.01688]
Model Saved


  0%|          | 0/4010 [00:00<?, ?it/s]

  0%|          | 0/1003 [00:00<?, ?it/s]

Epoch : [10] Train Loss : [0.01676] Val Loss : [0.01679]
Model Saved


In [13]:
SAVE_PATH = os.getcwd() + '/lstm_2layers.pth'
torch.save(infer_model.state_dict(), SAVE_PATH)

train_loss = np.array(train_loss)
val_loss = np.array(val_loss)

# loss값 저장
np.save(PATH + 'train_loss_lstm_2layers', train_loss)
np.save(PATH + 'val_loss_lstm_2layers', val_loss)

In [16]:
test_input = np.load(LOADPATH + 'test_input_minmax.npy')
min_ = np.load(PATH + 'MinMax/MIN.npy')
max_ = np.load(PATH + 'MinMax/MAX.npy')

In [17]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [ ]:
model = LSTM()
if platform.system() == "Darwin":
    model.load_state_dict(torch.load(os.getcwd() + '/cnn_lstm_drop_0.4_1e_4.pth', map_location=torch.device('cpu')))
else:
    model.load_state_dict(torch.load(os.getcwd() + '/cnn_lstm_drop_0.4_1e_4.pth'))

### Prediction

In [18]:
prediction = inference(infer_model, test_loader, device)
prediction.shape

  0%|          | 0/16 [00:00<?, ?it/s]

(15890, 21)

In [19]:
def return_scaler(arr, min_ = min_, max_ = max_):
    denom = max_ - min_
    for i in tqdm(range(len(arr))):
        arr[i] = (arr[i]*denom[i]) + min_[i]
    
    return arr

prediction_unscale = np.around(return_scaler(prediction))
prediction_unscale

  0%|          | 0/15890 [00:00<?, ?it/s]

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 2., 2., 2.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [2., 2., 2., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
submit = pd.read_csv(PATH + '/sample_submission.csv') # submit은 15690개 상품 / 예측은 15682개 상품
submit.iloc[:, 1:] = prediction_unscale
submit

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,0,0,0,0,0,0,1,1,1,...,1,1,1,1,1,2,2,2,2,2
3,3,0,0,0,0,1,1,1,1,2,...,2,2,2,2,3,3,3,3,3,3
4,4,1,3,3,3,3,4,4,5,5,...,5,5,5,6,6,6,6,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,0,3,3,3,3,3,3,4,4,...,4,4,4,4,5,4,4,4,4,4
15886,15886,4,6,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
15887,15887,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
15888,15888,2,2,2,2,1,1,1,2,2,...,1,1,1,2,2,2,1,1,1,1


In [21]:
SAVE_SUBMIT = os.getcwd() + '/data/lstm_2Layers.csv'
submit.to_csv(SAVE_SUBMIT, index_label=False)